In [60]:
import sys
sys.path.append('/opt/ml')

In [70]:
import pandas as pd
import numpy as np
import os
import pickle
from utils import data_controller

In [62]:
ensemble_csvs_dir = "./ingredients"
print(os.listdir(ensemble_csvs_dir))

['05-16_14:27:21_CJW_23_EP5_bs64_warmup0.1_LR3e-5_tml160_val15_RD_APNS_QP_FocLoss_LSTMNT____86.7762_.csv', '05-16_15:04:09_CJW_23_EP5_bs64_warmup0.1_LR3e-5_tml160_val15_RD_NC_APNS_QP_LabSmth_LSTMNT____87.2437_.csv', '05-16_23:27:41_CJW_BERT_23_EP1_bs64_LR3e-5_tml160_noVal_RD_APNS_QP_LabSmth_LSTMNT____84.2351_.csv']


In [63]:
test_dataset = pd.read_csv("../dataset/new_test.csv")

test_id = list(test_dataset['id']) 

In [64]:
prob_list = []
weights = []

for csv in os.listdir(ensemble_csvs_dir):
    if csv[0] != '.':
        # Extract the score from the filename and append it to the weights
        score = float(csv.split('_')[-2])
        weights.append(score)

        # Load the probabilities and append them to the list
        prob = [eval(x) for x in pd.read_csv(os.path.join(ensemble_csvs_dir,csv))["probs"]]
        prob_list.append(prob)

# Convert the lists to NumPy arrays
new_probs = np.array(prob_list)
weights = np.array(weights)

print(new_probs.shape)
print(weights.shape)

(3, 7765, 30)
(3,)


In [65]:
# Calculate the weighted average of the probabilities
weighted_avg_probs = np.sum(new_probs * weights[:, None, None], axis=0) / np.sum(weights)

print(weighted_avg_probs.shape)
print(weighted_avg_probs)
print("Total to:", np.sum(weighted_avg_probs[0:30], axis=1))

(7765, 30)
[[0.14042456 0.00510988 0.03538339 ... 0.00192084 0.00328135 0.00237828]
 [0.10781213 0.00479456 0.00287686 ... 0.00287671 0.00260704 0.00207379]
 [0.92870803 0.00194232 0.0021319  ... 0.00226959 0.00202293 0.00210774]
 ...
 [0.02328038 0.91375961 0.00227886 ... 0.00212933 0.00380704 0.00212323]
 [0.84183117 0.0014515  0.00152172 ... 0.00185327 0.00239037 0.00248169]
 [0.93804209 0.00176347 0.0015144  ... 0.00176434 0.00168871 0.00200606]]
Total to: [1.00000006 0.99999999 1.00000005 0.99999996 1.00000009 1.00000001
 1.00000002 1.00000001 1.         0.99999997 0.99999998 0.99999996
 0.99999996 1.00000007 0.99999999 0.99999999 0.99999995 0.99999995
 0.99999992 0.99999998 1.00000002 1.         1.00000003 1.
 1.00000004 1.         0.99999996 1.00000005 0.99999999 1.00000004]


In [66]:
pred_answer = np.argmax(weighted_avg_probs, axis=-1)
print(pred_answer.shape)

(7765,)


In [76]:
num2label = data_controller.load_num2label()
label_answer = [num2label[i] for i in pred_answer]
print(len(pred_answer))

7765


In [77]:
output = pd.DataFrame({'id':test_id,'pred_label':label_answer,'probs':weighted_avg_probs.tolist(),})
output.to_csv('./food/ensembled_output.csv', index=False) 